# Natural Orbital Functional Theory: Dynamic (Scheduled) step size

Import libreries

In [7]:
import pynof
from scipy.linalg import eigh
import numpy as np
import csv
import sys
# Define energy.py path, in our case is in myCode dir, two directories above
sys.path.insert(1, '../../myCode/')
# read files with optimization functions
from energy import *

Create molecule and choose basis set

In [10]:
#Molecules Definition
#Times given in Seconds, gotten on a Linux PC (12th Gen Intel(R) Core(TM) i9-12900K)
#with export OMP_NUM_THREADS=8
#time:  326 
co2 = pynof.molecule("""
0 1
C    0.0000    0.0000    0.0000   
O    1.1000    0.0000    0.0000   
O    -1.1000   -0.0000    0.0000 
""")

#time: 2236
bf3 = pynof.molecule("""
0 1
 F     4.007177     0.908823     0.000000
 F     1.724922     0.908844     0.000000
 F     2.866001    -1.067658    -0.000000
 B     2.866001     0.249991    -0.000000
""")

#time: 9185
c4h4 = pynof.molecule("""
 0 1
C          0.71430       -0.67320        0.00020
C          0.71480        0.67270       -0.00020
C         -0.71490       -0.67260       -0.00020
C         -0.71420        0.67320        0.00020
H          1.46990       -1.44980        0.00030
H          1.47100        1.44850       -0.00030
H         -1.47110       -1.44850       -0.00030
H         -1.46980        1.44960        0.00020
""")
#time: 281.21
c2h2 = pynof.molecule("""
 0 1
 C     -0.6000    0.0000    0.0000
 C      0.6000    0.0000    0.0000
 H     -1.6650    0.0000   -0.0001
 H      1.6650    0.0000    0.0001
""")

#time: 1334
c2h4 = pynof.molecule("""
 0 1
C   -0.6672    0.0000    0.0000
C    0.6672    0.0000    0.0000
H   -1.2213   -0.9290    0.0708
H   -1.2212    0.9290   -0.0708
H    1.2213    0.9290   -0.0708
H    1.2213   -0.9290    0.0708
""")

#time: 2881
hno3 = pynof.molecule("""
 0 1
    N       -0.151833    0.032380   -0.000040
    O       -1.021558   -0.782138    0.000011
    O        1.148151   -0.517360    0.000013
    O       -0.208610    1.237710    0.000014
    H        1.718969    0.267641   -0.000028
""")

#time: 466
o3 = pynof.molecule("""
 0 1
 O     1.068900     0.000000     0.215300
 O    -0.024957     0.000000    -0.445578
 O    -1.108044     0.000000     0.232808
 """)
# Dictionary to facilitate molecule selection with 
# output file name

molecules = {
    "bf3": bf3,
    "co2": co2,
    "hno3": hno3,
    "o3": o3,
}

######################
# USER ACTION REQUIRED
# Select molecule 
molecule = "o3" # or "co2" or "bf3" or "hno3"
######################
mol = molecules[molecule]

# Select basis set
basis = "cc-pvdz"
p = pynof.param(mol,basis)


   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1-3 entry O          line   198 file /Users/pobmelat/miniconda3/envs/newConda/share/psi4/basis/cc-pvdz.gbs 



Run calculations with a dynamyc alpha value

In [ ]:
# Here we select the NOF functional. 8 should be fine
p.ipnof = 8
p.maxit = 80

p.orb_method="ADAM"

alpha_values = [0.001, 0.002, 0.005, 0.02, 0.04, 0.08]
#all_energy_data = {}  # Dictionary to store results for each alpha

# File name to store the data
filename = f"{molecule}_{"NOF"}.csv"

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["alpha", "iteration", "energy"])  # Header

    for alpha in alpha_values:
        p.alpha = alpha
        # Select molecule
        _, energy_data = calc_nof_orbrot(mol, p)
        for i_ext, energy in energy_data:
            writer.writerow([alpha, i_ext, energy])

    # ADAM algorithm including dynamic modification, ADAM2
    p.orb_method="ADAM2"
    p.alpha = 0.02
    _, energy_data = calc_nof_orbrot(mol, p)
    for i_ext, energy in energy_data:
        writer.writerow([0.0, i_ext, energy])

Number of basis functions                   (NBF)    = 42
Inactive Doubly occupied orbitals up too     (NO1)    = 0
No. considered Strongly Doubly occupied MOs (NDOC)   = 12
No. considered Strongly Singly occupied MOs (NSOC)   = 0
No. of Weakly occ. per St. Doubly occ.  MOs (NCWO)   = 2
Dimension of the Nat. Orb. subspace         (NBF5)   = 36
No. of electrons                                     = 24
Multiplicity                                         = 1

No violations of the orthonormality

PNOF8 Calculation (ADAM/Softmax Optimization)

Nitext  Nit_orb  Nit_occ      Eelec           Etot          Ediff        Grad_orb   Grad_occ Conv Orb Conv Occ
alpha =  0.001
     0     30     42    -282.65389351  -214.09657477 -10281.65389351      1.1e+01    3.0e-02   False   True
alpha =  0.001
     1     30    128    -285.10839167  -216.55107292     -2.45449816      9.1e+00    1.4e-05   False   True
alpha =  0.001
     2     30     77    -286.99401778  -218.43669904     -1.88562611      8.1e+00 